<a href="https://colab.research.google.com/github/yousenwang/langchain_llm/blob/main/experiments/2023/08/14/step2%20LLaMA2_7B_14gbVRam_k3_1000_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.

In [2]:
!nvidia-smi

Mon Aug 14 07:19:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from huggingface_hub import notebook_login

notebook_login()


## LLaMA2 7B Chat


In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:628: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [6]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [7]:
!nvidia-smi

Mon Aug 14 07:22:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    26W /  70W |  13801MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [40]:
prompt2 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  6 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure
5. Submit the RMA  request

⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit
⚫ Description: Submit the RMA request

The Reminder field will show you whether this RMA request is approvable
(acceptable)  for warranty provider  or not.
Please remember to fill in WAL# in Customer No. field
Lastly, click Submit  in the top right corner to finish the  RMA requesting process2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return

  Question: What does the Reminder field indicate when submitting an RMA request?
  Answer:
"""

In [29]:

def llama_generate(prompt):
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs,
                            max_new_tokens=512,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.eos_token_id,
                            )
  final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
  final_outputs = cut_off_text(final_outputs, '</s>')
  final_outputs = remove_substring(final_outputs, prompt)
  return final_outputs

# Ans2: The Reminder field shows whether the RMA request is approvable (acceptable) for the warranty provider or not.

In [41]:
print(llama_generate(prompt2))

The Reminder field will show you whether this RMA request is approvable (acceptable) for warranty provider or not.


In [47]:
prompt4 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return6 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure
5. Submit the RMA  request

⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit
⚫ Description: Submit the RMA request

The Reminder field will show you whether this RMA request is approvable
(acceptable)  for warranty provider  or not.
Please remember to fill in WAL# in Customer No. field
Lastly, click Submit  in the top right corner to finish the  RMA requesting process

  Question: What steps should be followed when filling in the request information for an RMA?
  Answer:
"""

# Ans4: To fill in the request information for an RMA, follow these steps:
```
Fill in the Product SN (Serial Number), Product PN (Part Number), and defective reason.
Select the Rack PN (Part Number) by clicking on "Please select Parts" and choose the desired quantity.
Click on "Confirm" to proceed.
Optionally, fill in the Parts SN (Serial Number) for the failure part (not required).
Click on "Save" to save the information.

```

In [48]:
print(llama_generate(prompt4))

I don't know.


In [45]:
prompt5 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  1 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure






wareconn standard operating procedure
RMA request  SOP

Version :
Version  Date  Editor  Description
v 1.0  2023/03/16 Eric Sun First draft (English version)


Content
SOP ................................ ................................ ................................ ................................  2
1.1 RMA request procedure (Parts return)  ................................ ...............  24 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return3 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure
2. Go to  RMA Request  Page
⚫ Module: Customer portal -Warranty Claims -Warranty Claims
⚫ Description: Go to RMA request page

To go to the RMA request page, follow these steps:
Click Warranty Claims  in the left menu, then click +Add

Click the icon shown in the image below
(Select the corresponding SI )

  Question: Can this document be used for purposes other than introducing Wareconn functions and procedures?
  Answer:
"""

# Ans5: No, this document explicitly states that it should not be used for other purposes. Its sole intention is to introduce Wareconn functions and procedures.

In [46]:
print(llama_generate(prompt5))

I don't know.


In [35]:
prompt3 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  6 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure
5. Submit the RMA  request

⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit
⚫ Description: Submit the RMA request

The Reminder field will show you whether this RMA request is approvable
(acceptable)  for warranty provider  or not.
Please remember to fill in WAL# in Customer No. field
Lastly, click Submit  in the top right corner to finish the  RMA requesting process2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return

  Question: What information should be filled in the Customer No. field when submitting an RMA request?
  Answer:
"""

# Ans3: When submitting an RMA request, you should fill in the WAL# (Warranty Authorization Number) in the Customer No. field.

In [37]:
print(llama_generate(prompt3))

The Customer No. field should be filled in with the WAL#.


In [38]:
prompt1 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal3 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure
2. Go to  RMA Request  Page
⚫ Module: Customer portal -Warranty Claims -Warranty Claims
⚫ Description: Go to RMA request page

To go to the RMA request page, follow these steps:
Click Warranty Claims  in the left menu, then click +Add

Click the icon shown in the image below
(Select the corresponding SI )4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return

  Question: How can I navigate to the RMA request page in the Wareconn Customer Portal?
  Answer:
"""

In [44]:
print(llama_generate(prompt1))

To navigate to the RMA request page in the Wareconn Customer Portal, you can follow these steps:

1. Go to the Wareconn Customer Portal by clicking on the link provided in the document.
2. Once logged in, click on the Warranty Claims menu in the left menu.
3. Click on the +Add button next to Warranty Claims.
4. Select the corresponding SI (System Interface) and click Enter.

Alternatively, you can also navigate to the RMA request page by clicking on the RMA Request link in the top menu bar of the Customer Portal.


# A1: To navigate to the RMA request page, click on "Warranty Claims" in the left menu, then click on the "+Add" button.